<a href="https://colab.research.google.com/github/mirzarad/CSE308/blob/master/Cali_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
os.chdir("/content/spatialindex-src-1.8.5")
!./configure
!make
!make install
!ldconfig

In [0]:
!pip install geopandas
!pip install rtree
!pip install pyshp
!pip install pyproj
!pip install geopy
!pip install fiona
!pip install https://github.com/JosPolfliet/pandas-profiling/archive/master.zip

!pip install https://github.com/andrea-cuttone/geoplotlib/archive/master.zip

!pip install https://github.com/ResidentMario/missingno/archive/master.zip
!pip install maup

In [0]:
#import some packages you will be using
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
from scipy import stats
from sklearn import linear_model, metrics 
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [88]:
from google.colab import files
uploaded=files.upload()

Saving cali_demogr.csv to cali_demogr (1).csv


In [0]:
!unzip /content/spatialindex-src-1.8.5/srprec_state_g16_v01_shp.zip
!unzip /content/spatialindex-src-1.8.5/state_g16_sov_data_by_g16_srprec.zip
!unzip /content/spatialindex-src-1.8.5/PL9410_BLOCK10_PL94_2_4_H1.zip


In [0]:
# making data frame from csv file
sprec_to_blk= pd.read_csv("/content/spatialindex-src-1.8.5/state_g18_sr_blk_map.csv")
sprec_to_blk.head()
cali_elec_2016 = pd.read_csv("/content/spatialindex-src-1.8.5/state_g18_sov_data_by_g18_srprec.csv")


In [34]:
sprec_to_blk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398568 entries, 0 to 398567
Data columns (total 14 columns):
COUNTY        398568 non-null int64
FIPS          398568 non-null int64
ELECTION      398568 non-null object
TYPE          398568 non-null object
SRPREC_KEY    398568 non-null object
BLOCK_KEY     398568 non-null int64
SRPREC        398534 non-null object
TRACT         398568 non-null int64
BLOCK         398568 non-null int64
BLKREG        398568 non-null int64
SRTOTREG      398534 non-null float64
PCTSRPREC     398534 non-null float64
BLKTOTREG     398568 non-null int64
PCTBLK        398568 non-null float64
dtypes: float64(3), int64(7), object(4)
memory usage: 42.6+ MB


In [35]:
sprec_to_blk['BLOCK_KEY'].nunique()

382395

In [36]:
sprec_to_blk.drop(["COUNTY", "FIPS", 	"ELECTION", "TYPE","TRACT","BLKREG",	"SRTOTREG", 	"PCTSRPREC", 	"BLKTOTREG"],axis=1,inplace=True)
sprec_to_blk.head()

,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK,PCTBLK
0,06097nan,60970000000000,NaN,0,100.0
1,060971002,60971525011014,1002,1014,100.0
2,060971002,60971525013008,1002,3008,100.0
3,060971006,60971522025016,1006,5016,100.0
4,060971008,60971526001002,1008,1002,100.0


In [0]:
sprec_to_blk.duplicated(["BLOCK_KEY","SRPREC"])

In [39]:
!pip install simpledbf
from simpledbf import Dbf5
dbf = Dbf5('/content/spatialindex-src-1.8.5/PL9410_BLOCK10_PL94_2_4_H1.DBF')
cencus = dbf.to_dataframe()

In [0]:
cencus.drop(["FILEID" ,	"STATE",	"SUMLEV" ,"LOGRECNO",	"REGION", "REGION", "DIVISION" ,
           "COUSUB",	"COUSUBCC","PLACE","PLACECC","TRACT","BLKGROUP","CD01","GCUNI",
           "H0010001", 	"H0010002", 	"H0010003","V21", 	"V31", 	"V41","V51","V11"],axis=1,
          inplace=True)


In [41]:
cencus.head()

,COUNTYFIPS,BLOCK_KEY,BLOCK,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,P0020031,P0020032,P0020033,P0020034,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,06001,060014277003018,3018,Block 3018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,06001,060014283014046,4046,Block 4046,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,06001,060014284001020,1020,Block 1020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,06001,060014285001009,1009,Block 1009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,06001,060014286002004,2004,Block 2004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
cencus["BLOCK_KEY"].nunique()

710145

In [43]:
cencus.describe()

,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,P0020031,P0020032,P0020033,P0020034,P0020035,P0020036,P0020037,P0020038,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
count,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.00000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,...,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.000000,710145.00000,710145.000000,710145.000000,710145.000000,710145.000000
mean,52.459647,52.459647,52.459647,19.733602,32.726045,31.361963,21.060844,3.046989,0.228474,6.724077,0.181057,0.12052,1.364082,1.251071,0.205949,0.196525,0.552539,0.045119,0.031927,0.036633,0.051896,0.009231,0.011562,0.014406,0.001977,0.001752,0.050345,0.036618,0.004592,0.102548,0.033269,0.015193,0.002463,0.002387,0.011350,0.001901,0.001317,0.022976,0.003467,0.000411,...,0.001046,0.000969,0.012965,0.002124,0.000291,0.001257,0.000221,0.000569,0.000773,0.000408,0.000159,0.000404,0.000117,0.000030,0.000796,0.004985,0.002366,0.000318,0.000682,0.000332,0.000135,0.000027,0.000738,0.000073,0.000025,0.000093,0.000099,0.000042,0.000013,0.000031,0.000011,0.000715,0.000600,0.000066,0.000015,0.00001,0.000017,0.000007,0.000055,0.000055
std,120.585772,120.585772,120.585772,60.870380,80.981718,77.578813,52.376714,18.196877,1.556411,28.455181,1.359181,1.31208,4.232405,3.901762,0.945852,0.833577,2.176759,0.348542,0.268467,0.334335,0.390448,0.155505,0.204175,0.195308,0.065870,0.054326,0.427623,0.384591,0.130608,0.564087,0.296978,0.184933,0.072578,0.067033,0.155469,0.063322,0.049439,0.235961,0.081487,0.028477,...,0.039110,0.037287,0.143952,0.057780,0.022607,0.041139,0.019096,0.029207,0.032898,0.025337,0.017318,0.023995,0.014289,0.008474,0.042992,0.089861,0.062613,0.023312,0.033346,0.020275,0.012780,0.006166,0.031207,0.010208,0.007690,0.011004,0.012780,0.007690,0.003560,0.009640,0.004440,0.034914,0.032449,0.009713,0.003936,0.00356,0.004440,0.003140,0.009713,0.009713
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [0]:
cencus.duplicated()

In [31]:
cencus.dtypes

COUNTYFIPS    object
BLOCK_KEY     object
BLOCK         object
NAME          object
POPULATION     int64
               ...  
P0040069       int64
P0040070       int64
P0040071       int64
P0040072       int64
P0040073       int64
Length: 153, dtype: object

In [0]:
census = cencus.iloc[0:100000]

In [45]:
census.head()

,COUNTYFIPS,BLOCK_KEY,BLOCK,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,P0020031,P0020032,P0020033,P0020034,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,06001,060014277003018,3018,Block 3018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,06001,060014283014046,4046,Block 4046,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,06001,060014284001020,1020,Block 1020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,06001,060014285001009,1009,Block 1009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,06001,060014286002004,2004,Block 2004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
demographics1 = pd.merge(sprec_to_blk, census.astype({'BLOCK_KEY': 'int64'}), on='BLOCK_KEY')

In [47]:
demographics1.head()

,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK_x,PCTBLK,COUNTYFIPS,BLOCK_y,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,060231CS-1,60230007002004,1CS-1,2004,100.000000,06023,2004,Block 2004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,060231CS-1,60230007002005,1CS-1,2005,100.000000,06023,2005,Block 2005,53,53,53,0,53,53,47,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,060231CS-1,60230007002006,1CS-1,2006,100.000000,06023,2006,Block 2006,147,147,147,17,130,120,102,0,9,9,0,0,10,9,1,5,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,060231CS-1,60230007002008,1CS-1,2008,35.042735,06023,2008,Block 2008,184,184,184,21,163,156,149,0,3,4,0,0,7,7,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,060231CS-2,60230007002008,1CS-2,2008,64.957265,06023,2008,Block 2008,184,184,184,21,163,156,149,0,3,4,0,0,7,7,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
demographics1.isnull().sum()

SRPREC_KEY    0
BLOCK_KEY     0
SRPREC        0
BLOCK_x       0
PCTBLK        0
             ..
P0040069      0
P0040070      0
P0040071      0
P0040072      0
P0040073      0
Length: 157, dtype: int64

In [0]:
census = cencus.iloc[100000:200000]

In [0]:
demographics2 = pd.merge(sprec_to_blk, census.astype({'BLOCK_KEY': 'int64'}), on='BLOCK_KEY')

In [51]:
demographics2.head()

,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK_x,PCTBLK,COUNTYFIPS,BLOCK_y,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,06033111,60330011001173,111,1173,98.039216,06033,1173,Block 1173,24,24,24,2,22,21,21,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,06033581,60330011001173,581,1173,1.960784,06033,1173,Block 1173,24,24,24,2,22,21,21,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,06033111,60330013001054,111,1054,100.000000,06033,1054,Block 1054,53,53,53,0,53,53,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,06033111,60330013001056,111,1056,100.000000,06033,1056,Block 1056,5,5,5,0,5,5,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,06033111,60330013001057,111,1057,100.000000,06033,1057,Block 1057,22,22,22,0,22,22,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
demographics1.isnull().sum()

SRPREC_KEY    0
BLOCK_KEY     0
SRPREC        0
BLOCK_x       0
PCTBLK        0
             ..
P0040069      0
P0040070      0
P0040071      0
P0040072      0
P0040073      0
Length: 157, dtype: int64

In [0]:
census = cencus.iloc[200000:300000]

In [0]:
demographics3 = pd.merge(sprec_to_blk, census.astype({'BLOCK_KEY': 'int64'}), on='BLOCK_KEY')

In [55]:
demographics3.head()

,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK_x,PCTBLK,COUNTYFIPS,BLOCK_y,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,0604701001,60470003011000,01001,1000,100.0,06047,1000,Block 1000,353,353,353,250,103,101,25,5,1,70,0,0,2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0604701001,60470003011005,01001,1005,100.0,06047,1005,Block 1005,172,172,172,125,47,43,12,0,0,31,0,0,4,4,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0604701001,60470003011006,01001,1006,100.0,06047,1006,Block 1006,162,162,162,137,25,25,15,3,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0604701001,60470003011015,01001,1015,100.0,06047,1015,Block 1015,14,14,14,13,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0604701001,60470003011017,01001,1017,100.0,06047,1017,Block 1017,37,37,37,30,7,7,0,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
census = cencus.iloc[300000:500000]
demographics4 = pd.merge(sprec_to_blk, census.astype({'BLOCK_KEY': 'int64'}), on='BLOCK_KEY')

In [57]:
demographics4.head()

,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK_x,PCTBLK,COUNTYFIPS,BLOCK_y,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,06057CP01,60570002002009,CP01,2009,25.490196,06057,2009,Block 2009,76,76,76,4,72,72,63,4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,06057CP02,60570002002009,CP02,2009,56.862745,06057,2009,Block 2009,76,76,76,4,72,72,63,4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,06057CP18,60570002002009,CP18,2009,17.647059,06057,2009,Block 2009,76,76,76,4,72,72,63,4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,06057CP01,60570002002025,CP01,2025,78.125000,06057,2025,Block 2025,134,134,134,2,132,131,128,0,3,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,06057CP17,60570002002025,CP17,2025,21.875000,06057,2025,Block 2025,134,134,134,2,132,131,128,0,3,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
census = cencus.iloc[500000:]
demographics5 = pd.merge(sprec_to_blk, census.astype({'BLOCK_KEY': 'int64'}), on='BLOCK_KEY')

In [0]:
frames = [demographics1, demographics2, demographics3,demographics4,demographics5]
demographics = pd.concat(frames)

In [60]:
demographics.head()

,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK_x,PCTBLK,COUNTYFIPS,BLOCK_y,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,060231CS-1,60230007002004,1CS-1,2004,100.000000,06023,2004,Block 2004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,060231CS-1,60230007002005,1CS-1,2005,100.000000,06023,2005,Block 2005,53,53,53,0,53,53,47,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,060231CS-1,60230007002006,1CS-1,2006,100.000000,06023,2006,Block 2006,147,147,147,17,130,120,102,0,9,9,0,0,10,9,1,5,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,060231CS-1,60230007002008,1CS-1,2008,35.042735,06023,2008,Block 2008,184,184,184,21,163,156,149,0,3,4,0,0,7,7,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,060231CS-2,60230007002008,1CS-2,2008,64.957265,06023,2008,Block 2008,184,184,184,21,163,156,149,0,3,4,0,0,7,7,2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
demographics.dtypes

SRPREC_KEY     object
BLOCK_KEY       int64
SRPREC         object
BLOCK_x         int64
PCTBLK        float64
               ...   
P0040069        int64
P0040070        int64
P0040071        int64
P0040072        int64
P0040073        int64
Length: 157, dtype: object

In [61]:
columns = demographics.columns[8:157]
import math
for i in columns:
  demographics[i] = round(((demographics[i]*demographics["PCTBLK"])/100),0)
demographics.head()



,SRPREC_KEY,BLOCK_KEY,SRPREC,BLOCK_x,PCTBLK,COUNTYFIPS,BLOCK_y,NAME,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
0,060231CS-1,60230007002004,1CS-1,2004,100.000000,06023,2004,Block 2004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,060231CS-1,60230007002005,1CS-1,2005,100.000000,06023,2005,Block 2005,53.0,53.0,53.0,0.0,53.0,53.0,47.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,060231CS-1,60230007002006,1CS-1,2006,100.000000,06023,2006,Block 2006,147.0,147.0,147.0,17.0,130.0,120.0,102.0,0.0,9.0,9.0,0.0,0.0,10.0,9.0,1.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,060231CS-1,60230007002008,1CS-1,2008,35.042735,06023,2008,Block 2008,64.0,64.0,64.0,7.0,57.0,55.0,52.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,060231CS-2,60230007002008,1CS-2,2008,64.957265,06023,2008,Block 2008,120.0,120.0,120.0,14.0,106.0,101.0,97.0,0.0,2.0,3.0,0.0,0.0,5.0,5.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
demographics["SRPREC"].nunique()

20373

In [0]:
cali_demogr=demographics.groupby("SRPREC").sum()

In [0]:
cali_demogr.drop(["BLOCK_KEY","BLOCK_x","PCTBLK"],axis=1,inplace=True)

In [65]:
cali_demogr.head()

,POPULATION,P0010001,P0020001,P0020002,P0020003,P0020004,P0020005,P0020006,P0020007,P0020008,P0020009,P0020010,P0020011,P0020012,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,P0020031,P0020032,P0020033,P0020034,P0020035,P0020036,P0020037,P0020038,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
SRPREC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,13666.0,13666.0,13666.0,5853.0,7813.0,7551.0,5279.0,879.0,219.0,1138.0,13.0,21.0,264.0,251.0,47.0,103.0,48.0,9.0,9.0,5.0,7.0,1.0,0.0,4.0,0.0,3.0,7.0,9.0,0.0,12.0,3.0,2.0,2.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,7108.0,7108.0,7108.0,2733.0,4375.0,4196.0,2825.0,602.0,109.0,643.0,10.0,9.0,177.0,168.0,21.0,58.0,45.0,6.0,5.0,2.0,3.0,0.0,1.0,2.0,6.0,1.0,0.0,17.0,0.0,8.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,6063.0,6063.0,6063.0,2468.0,3593.0,3450.0,2227.0,683.0,280.0,248.0,5.0,8.0,143.0,134.0,30.0,46.0,23.0,2.0,3.0,10.0,5.0,0.0,8.0,2.0,0.0,1.0,0.0,3.0,0.0,9.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,8581.0,8581.0,8581.0,2537.0,6045.0,5814.0,4123.0,581.0,155.0,912.0,25.0,17.0,230.0,219.0,34.0,70.0,62.0,10.0,5.0,5.0,11.0,0.0,1.0,6.0,0.0,0.0,5.0,10.0,0.0,12.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,10388.0,10388.0,10388.0,3890.0,6497.0,6231.0,4006.0,1060.0,138.0,992.0,26.0,12.0,264.0,244.0,42.0,75.0,65.0,3.0,0.0,10.0,7.0,4.0,0.0,7.0,0.0,0.0,16.0,16.0,0.0,18.0,11.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
cali_demogr.to_csv("cali_preproc_demogr2018.csv")

In [67]:
cali_demogr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20373 entries, 1.0 to YV52128
Columns: 149 entries, POPULATION to P0040073
dtypes: float64(149)
memory usage: 23.3+ MB


In [0]:
cali_demographics = cali_demogr.rename(columns = {"P0010001":"TAPERSONS","P0020002":"TAHISPANIC","P0020003":"TANHPOALN"
,"P0020004":"TA1RACE","P0020005":"TAWHITEALN", 	"P0020006":"TABLACKALN","P0020007":"TAAMINDALN", 	"P0020008":"TAASIANALN", 	"P0020009":"TANHPOALN", 
"P0020010":"TAOTHERALN","P0020012":"TA2RACE","P0040001":"VAPERSONS", 	"P0040002":"VAHISPANIC", 	"P0040003":"VANHPOALN" ,"P0040004":"VA1RACE",	"P0040005":"VAWHITEALN","P0040006":"VABLACKALN",
"P0040007":"VAAIANALN","P0040008":"VAASIANALN","P0040009":"VANHPOALN","P0040010":"VAOTHERALN"})

In [81]:
cali_demographics.head()

,POPULATION,TAPERSONS,P0020001,TAHISPANIC,TANHPOALN,TA1RACE,TAWHITEALN,TABLACKALN,TAAMINDALN,TAASIANALN,TANHPOALN,TAOTHERALN,P0020011,TA2RACE,P0020013,P0020014,P0020015,P0020016,P0020017,P0020018,P0020019,P0020020,P0020021,P0020022,P0020023,P0020024,P0020025,P0020026,P0020027,P0020028,P0020029,P0020030,P0020031,P0020032,P0020033,P0020034,P0020035,P0020036,P0020037,P0020038,...,P0040034,P0040035,P0040036,P0040037,P0040038,P0040039,P0040040,P0040041,P0040042,P0040043,P0040044,P0040045,P0040046,P0040047,P0040048,P0040049,P0040050,P0040051,P0040052,P0040053,P0040054,P0040055,P0040056,P0040057,P0040058,P0040059,P0040060,P0040061,P0040062,P0040063,P0040064,P0040065,P0040066,P0040067,P0040068,P0040069,P0040070,P0040071,P0040072,P0040073
SRPREC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,13666.0,13666.0,13666.0,5853.0,7813.0,7551.0,5279.0,879.0,219.0,1138.0,13.0,21.0,264.0,251.0,47.0,103.0,48.0,9.0,9.0,5.0,7.0,1.0,0.0,4.0,0.0,3.0,7.0,9.0,0.0,12.0,3.0,2.0,2.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,7108.0,7108.0,7108.0,2733.0,4375.0,4196.0,2825.0,602.0,109.0,643.0,10.0,9.0,177.0,168.0,21.0,58.0,45.0,6.0,5.0,2.0,3.0,0.0,1.0,2.0,6.0,1.0,0.0,17.0,0.0,8.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,6063.0,6063.0,6063.0,2468.0,3593.0,3450.0,2227.0,683.0,280.0,248.0,5.0,8.0,143.0,134.0,30.0,46.0,23.0,2.0,3.0,10.0,5.0,0.0,8.0,2.0,0.0,1.0,0.0,3.0,0.0,9.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,8581.0,8581.0,8581.0,2537.0,6045.0,5814.0,4123.0,581.0,155.0,912.0,25.0,17.0,230.0,219.0,34.0,70.0,62.0,10.0,5.0,5.0,11.0,0.0,1.0,6.0,0.0,0.0,5.0,10.0,0.0,12.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,10388.0,10388.0,10388.0,3890.0,6497.0,6231.0,4006.0,1060.0,138.0,992.0,26.0,12.0,264.0,244.0,42.0,75.0,65.0,3.0,0.0,10.0,7.0,4.0,0.0,7.0,0.0,0.0,16.0,16.0,0.0,18.0,11.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
cali_demographics = cali_demographics[["TAPERSONS","TAHISPANIC","TANHPOALN","TA1RACE","TAWHITEALN","TABLACKALN","TAAMINDALN","TAASIANALN","TANHPOALN", 
"TAOTHERALN","TA2RACE","VAPERSONS","VAHISPANIC","VANHPOALN","VA1RACE","VAWHITEALN","VABLACKALN",
"VAAIANALN","VAASIANALN","VANHPOALN","VAOTHERALN"]]

In [86]:
cali_demographics.head()

,TAPERSONS,TAHISPANIC,TANHPOALN,TANHPOALN,TANHPOALN,TANHPOALN,TA1RACE,TAWHITEALN,TABLACKALN,TAAMINDALN,TAASIANALN,TANHPOALN,TANHPOALN,TANHPOALN,TANHPOALN,TAOTHERALN,TA2RACE,VAPERSONS,VAHISPANIC,VANHPOALN,VANHPOALN,VANHPOALN,VANHPOALN,VA1RACE,VAWHITEALN,VABLACKALN,VAAIANALN,VAASIANALN,VANHPOALN,VANHPOALN,VANHPOALN,VANHPOALN,VAOTHERALN
SRPREC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,13666.0,5853.0,7813.0,13.0,7813.0,13.0,7551.0,5279.0,879.0,219.0,1138.0,7813.0,13.0,7813.0,13.0,21.0,251.0,9341.0,3484.0,5857.0,11.0,5857.0,11.0,5709.0,4200.0,601.0,158.0,722.0,5857.0,11.0,5857.0,11.0,16.0
2.0,7108.0,2733.0,4375.0,10.0,4375.0,10.0,4196.0,2825.0,602.0,109.0,643.0,4375.0,10.0,4375.0,10.0,9.0,168.0,4928.0,1692.0,3237.0,6.0,3237.0,6.0,3146.0,2178.0,465.0,74.0,416.0,3237.0,6.0,3237.0,6.0,8.0
3.0,6063.0,2468.0,3593.0,5.0,3593.0,5.0,3450.0,2227.0,683.0,280.0,248.0,3593.0,5.0,3593.0,5.0,8.0,134.0,4142.0,1450.0,2691.0,5.0,2691.0,5.0,2619.0,1768.0,507.0,183.0,153.0,2691.0,5.0,2691.0,5.0,4.0
4.0,8581.0,2537.0,6045.0,25.0,6045.0,25.0,5814.0,4123.0,581.0,155.0,912.0,6045.0,25.0,6045.0,25.0,17.0,219.0,6189.0,1594.0,4595.0,24.0,4595.0,24.0,4464.0,3227.0,431.0,124.0,646.0,4595.0,24.0,4595.0,24.0,10.0
5.0,10388.0,3890.0,6497.0,26.0,6497.0,26.0,6231.0,4006.0,1060.0,138.0,992.0,6497.0,26.0,6497.0,26.0,12.0,244.0,6904.0,2153.0,4750.0,16.0,4750.0,16.0,4592.0,3179.0,695.0,86.0,610.0,4750.0,16.0,4750.0,16.0,7.0


In [0]:
cali_demographics.to_csv("cali_demogr.csv")

In [102]:
cali_elec_2016.head()

,COUNTY,SRPREC,ADDIST,CDDIST,SDDIST,BEDIST,TOTREG,DEMREG,REPREG,AIPREG,GRNREG,LIBREG,NLPREG,REFREG,DCLREG,MSCREG,TOTVOTE,DEMVOTE,REPVOTE,AIPVOTE,GRNVOTE,LIBVOTE,NLPVOTE,REFVOTE,DCLVOTE,MSCVOTE,PRCVOTE,ABSVOTE,ASSDEM01,ASSDEM02,ASSIND01,ASSLIB01,ASSREP01,ATGDEM01,ATGREP01,BOEDEM01,BOEREP01,CNGDEM01,CNGDEM02,CNGGRN01,...,GOVDEM01,GOVREP01,INSDEM01,INSIND01,LTGDEM01,LTGDEM02,PR_10_N,PR_10_Y,PR_11_N,PR_11_Y,PR_12_N,PR_12_Y,PR_1_N,PR_1_Y,PR_2_N,PR_2_Y,PR_3_N,PR_3_Y,PR_4_N,PR_4_Y,PR_5_N,PR_5_Y,PR_6_N,PR_6_Y,PR_7_N,PR_7_Y,PR_8_N,PR_8_Y,SENDEM01,SENDEM02,SENIND01,SENREP01,SOSDEM01,SOSREP01,SPINOP01,SPINOP02,TRSDEM01,TRSREP01,USSDEM01,USSDEM02
0,49,1002,2,5,2,2,28,0,0,0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,20,0,0,0,4,19,6,19,6,20,0,0,...,18,7,17,8,5,20,10,13,10,14,8,17,9,16,7,17,12,12,15,9,18,6,17,7,12,13,10,12,17,6,0,0,21,4,10,11,19,6,8,17
1,49,1006,2,5,2,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,...,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,1,0,1,0
2,49,1008,2,5,2,2,152,0,0,0,0,0,0,0,0,0,127,0,0,0,0,0,0,0,0,0,0,0,83,0,0,0,33,87,35,81,35,92,0,0,...,87,37,68,48,33,74,64,62,62,60,37,90,48,76,41,82,59,65,53,70,79,45,98,29,55,68,65,57,63,48,0,0,86,33,38,67,84,33,58,58
3,49,1009,2,5,2,2,147,0,0,0,0,0,0,0,0,0,117,0,0,0,0,0,0,0,0,0,0,0,73,0,0,0,39,78,38,73,38,83,0,0,...,81,35,57,51,29,69,66,50,38,75,40,72,44,67,33,80,47,62,44,66,75,39,84,32,48,68,63,48,58,41,0,0,75,40,39,54,78,38,45,57
4,49,1012,2,5,2,2,22,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,4,13,2,13,2,14,0,0,...,14,2,10,6,3,12,7,6,8,7,3,11,4,10,4,10,7,6,9,5,7,4,14,1,3,9,8,4,8,7,0,0,15,1,3,7,14,1,5,11


In [0]:
cali_elec_2016.drop(cali_elec_2016.iloc[:,38:72],axis=1,inplace=True)

In [226]:
cali_elec_2016.head()

,COUNTY,SRPREC,ADDIST,CDDIST,SDDIST,BEDIST,TOTREG,DEMREG,REPREG,AIPREG,GRNREG,LIBREG,NLPREG,REFREG,DCLREG,MSCREG,TOTVOTE,DEMVOTE,REPVOTE,AIPVOTE,GRNVOTE,LIBVOTE,NLPVOTE,REFVOTE,DCLVOTE,MSCVOTE,PRCVOTE,ABSVOTE,ASSDEM01,ASSDEM02,ASSIND01,ASSLIB01,ASSREP01,ASSREP02,CNGDEM01,CNGDEM02,CNGIND01,CNGREP01,PRSDEM01,PRSGRN01,PRSLIB01,PRSPAF01,PRSREP01,SENDEM01,SENDEM02,SENLIB01,SENREP01,USSDEM01,USSDEM02
0,49,1001,2,5,2,2,244,0,0,0,0,0,0,0,0,0,219,0,0,0,0,0,0,0,0,0,0,0,147,0,0,49,0,0,168,0,0,44,141,5,12,1,50,0,0,0,0,148,33
1,49,1002,2,5,2,2,25,0,0,0,0,0,0,0,0,0,22,0,0,0,0,0,0,0,0,0,0,0,13,0,0,6,0,0,16,0,0,3,14,2,3,0,2,0,0,0,0,15,6
2,49,1006,2,5,2,2,4,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,2
3,49,1008,2,5,2,2,160,0,0,0,0,0,0,0,0,0,145,0,0,0,0,0,0,0,0,0,0,0,95,0,0,37,0,0,102,0,0,35,98,5,2,0,32,0,0,0,0,95,35
4,49,1009,2,5,2,2,146,0,0,0,0,0,0,0,0,0,129,0,0,0,0,0,0,0,0,0,0,0,79,0,0,33,0,0,88,0,0,31,80,6,4,1,32,0,0,0,0,74,32


In [0]:
cali_elec_2016.drop(cali_elec_2016.iloc[:,3:34],axis=1,inplace=True)

In [228]:
cali_elec_2016.head()

,COUNTY,SRPREC,ADDIST,CNGDEM01,CNGDEM02,CNGIND01,CNGREP01,PRSDEM01,PRSGRN01,PRSLIB01,PRSPAF01,PRSREP01,SENDEM01,SENDEM02,SENLIB01,SENREP01,USSDEM01,USSDEM02
0,49,1001,2,168,0,0,44,141,5,12,1,50,0,0,0,0,148,33
1,49,1002,2,16,0,0,3,14,2,3,0,2,0,0,0,0,15,6
2,49,1006,2,0,0,0,2,0,0,0,0,2,0,0,0,0,0,2
3,49,1008,2,102,0,0,35,98,5,2,0,32,0,0,0,0,95,35
4,49,1009,2,88,0,0,31,80,6,4,1,32,0,0,0,0,74,32


In [0]:
cali_elec_2016.drop(["ADDIST", 	"SENDEM01", 	"SENDEM02", 	"SENLIB01", 
                     "SENREP01" ,	"USSDEM01" ,	"USSDEM02"],axis=1,inplace=True)

In [230]:
cali_elec_2016.head()

,COUNTY,SRPREC,CNGDEM01,CNGDEM02,CNGIND01,CNGREP01,PRSDEM01,PRSGRN01,PRSLIB01,PRSPAF01,PRSREP01
0,49,1001,168,0,0,44,141,5,12,1,50
1,49,1002,16,0,0,3,14,2,3,0,2
2,49,1006,0,0,0,2,0,0,0,0,2
3,49,1008,102,0,0,35,98,5,2,0,32
4,49,1009,88,0,0,31,80,6,4,1,32


In [0]:
cali_elec_2016.drop(cali_elec_2016.loc[cali_elec_2016['SRPREC']=="CNTYTOT"].index, inplace=True)

In [0]:
cali_elec_2016["CNGDEM01"] = cali_elec_2016["CNGDEM01"] + cali_elec_2016["CNGDEM02"]

In [241]:

cali_elec_2016.drop(["CNGDEM02"],axis=1,inplace=True)
cali_elec_2016["TOTALCNG"] = cali_elec_2016["CNGDEM01"] + cali_elec_2016["CNGIND01"]+ cali_elec_2016["CNGREP01"]
cali_elec_2016["TOTALPRS"] = cali_elec_2016["PRSDEM01"]+cali_elec_2016["PRSGRN01"]+cali_elec_2016["PRSLIB01"]+cali_elec_2016["PRSPAF01"]+cali_elec_2016["PRSREP01"]
cali_elec_2016.head()

,COUNTY,SRPREC,CNGDEM01,CNGIND01,CNGREP01,PRSDEM01,PRSGRN01,PRSLIB01,PRSPAF01,PRSREP01,TOTALCNG,TOTALPRS
0,49,1001,168,0,44,141,5,12,1,50,212,209
1,49,1002,16,0,3,14,2,3,0,2,19,21
2,49,1006,0,0,2,0,0,0,0,2,2,2
3,49,1008,102,0,35,98,5,2,0,32,137,137
4,49,1009,88,0,31,80,6,4,1,32,119,123


In [0]:
cali_elec_2016 = cali_elec_2016.rename(columns={"CNGDEM01":"CNGDEM","CNGIND01":"CNGIND",	"CNGREP01":"CNGREP",	"PRSDEM01": "PRSDEM",	"PRSGRN01":"PRSGRN","PRSLIB01": "PRSLIB","PRSPAF01":"PRSPAF","PRSREP01":"PRSREP"})

In [245]:
cali_elec_2016.head()

,COUNTY,SRPREC,CNGDEM,CNGIND,CNGREP,PRSDEM,PRSGRN,PRSLIB,PRSPAF,PRSREP,TOTALCNG,TOTALPRS
0,49,1001,168,0,44,141,5,12,1,50,212,209
1,49,1002,16,0,3,14,2,3,0,2,19,21
2,49,1006,0,0,2,0,0,0,0,2,2,2
3,49,1008,102,0,35,98,5,2,0,32,137,137
4,49,1009,88,0,31,80,6,4,1,32,119,123


In [0]:
cali_elec_2016.to_csv("cali_elec_2018.csv")

In [0]:
# making data frame from csv file
dem= pd.read_csv("/content/spatialindex-src-1.8.5/cali_demogr (1).csv")
elec=pd.read_csv("/content/spatialindex-src-1.8.5/cali_elec_2016.csv")

In [90]:
dem.head()

,SRPREC,TAPERSONS,TAHISPANIC,TANHPOALN,TANHPOALN.1,TANHPOALN.2,TANHPOALN.3,TA1RACE,TAWHITEALN,TABLACKALN,TAAMINDALN,TAASIANALN,TANHPOALN.4,TANHPOALN.5,TANHPOALN.6,TANHPOALN.7,TAOTHERALN,TA2RACE,VAPERSONS,VAHISPANIC,VANHPOALN,VANHPOALN.1,VANHPOALN.2,VANHPOALN.3,VA1RACE,VAWHITEALN,VABLACKALN,VAAIANALN,VAASIANALN,VANHPOALN.4,VANHPOALN.5,VANHPOALN.6,VANHPOALN.7,VAOTHERALN
0,1.0,13666.0,5853.0,7813.0,13.0,7813.0,13.0,7551.0,5279.0,879.0,219.0,1138.0,7813.0,13.0,7813.0,13.0,21.0,251.0,9341.0,3484.0,5857.0,11.0,5857.0,11.0,5709.0,4200.0,601.0,158.0,722.0,5857.0,11.0,5857.0,11.0,16.0
1,2.0,7108.0,2733.0,4375.0,10.0,4375.0,10.0,4196.0,2825.0,602.0,109.0,643.0,4375.0,10.0,4375.0,10.0,9.0,168.0,4928.0,1692.0,3237.0,6.0,3237.0,6.0,3146.0,2178.0,465.0,74.0,416.0,3237.0,6.0,3237.0,6.0,8.0
2,3.0,6063.0,2468.0,3593.0,5.0,3593.0,5.0,3450.0,2227.0,683.0,280.0,248.0,3593.0,5.0,3593.0,5.0,8.0,134.0,4142.0,1450.0,2691.0,5.0,2691.0,5.0,2619.0,1768.0,507.0,183.0,153.0,2691.0,5.0,2691.0,5.0,4.0
3,4.0,8581.0,2537.0,6045.0,25.0,6045.0,25.0,5814.0,4123.0,581.0,155.0,912.0,6045.0,25.0,6045.0,25.0,17.0,219.0,6189.0,1594.0,4595.0,24.0,4595.0,24.0,4464.0,3227.0,431.0,124.0,646.0,4595.0,24.0,4595.0,24.0,10.0
4,5.0,10388.0,3890.0,6497.0,26.0,6497.0,26.0,6231.0,4006.0,1060.0,138.0,992.0,6497.0,26.0,6497.0,26.0,12.0,244.0,6904.0,2153.0,4750.0,16.0,4750.0,16.0,4592.0,3179.0,695.0,86.0,610.0,4750.0,16.0,4750.0,16.0,7.0


In [0]:
dem.drop(["TANHPOALN.1","TANHPOALN.2","TANHPOALN.3","TANHPOALN.4","TANHPOALN.5","TANHPOALN.6",
          "TANHPOALN.7","VANHPOALN.1","VANHPOALN.2","VANHPOALN.3","VANHPOALN.4","VANHPOALN.5","VANHPOALN.6","VANHPOALN.7"],axis=1,inplace=True)

In [92]:
dem.head()

,SRPREC,TAPERSONS,TAHISPANIC,TANHPOALN,TA1RACE,TAWHITEALN,TABLACKALN,TAAMINDALN,TAASIANALN,TAOTHERALN,TA2RACE,VAPERSONS,VAHISPANIC,VANHPOALN,VA1RACE,VAWHITEALN,VABLACKALN,VAAIANALN,VAASIANALN,VAOTHERALN
0,1.0,13666.0,5853.0,7813.0,7551.0,5279.0,879.0,219.0,1138.0,21.0,251.0,9341.0,3484.0,5857.0,5709.0,4200.0,601.0,158.0,722.0,16.0
1,2.0,7108.0,2733.0,4375.0,4196.0,2825.0,602.0,109.0,643.0,9.0,168.0,4928.0,1692.0,3237.0,3146.0,2178.0,465.0,74.0,416.0,8.0
2,3.0,6063.0,2468.0,3593.0,3450.0,2227.0,683.0,280.0,248.0,8.0,134.0,4142.0,1450.0,2691.0,2619.0,1768.0,507.0,183.0,153.0,4.0
3,4.0,8581.0,2537.0,6045.0,5814.0,4123.0,581.0,155.0,912.0,17.0,219.0,6189.0,1594.0,4595.0,4464.0,3227.0,431.0,124.0,646.0,10.0
4,5.0,10388.0,3890.0,6497.0,6231.0,4006.0,1060.0,138.0,992.0,12.0,244.0,6904.0,2153.0,4750.0,4592.0,3179.0,695.0,86.0,610.0,7.0


In [0]:
dem.to_csv("cali_demogr_2018.csv")